<a href="https://colab.research.google.com/github/Allenyang2/PytorchFL/blob/main/Flwr_Federated_Learning-working-changes-04-27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flwr[simulation] torch torchvision matplotlib

In [2]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import sys
import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.0.0+cu118 and Flower 1.4.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

In [3]:
CLASSES = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

We simulate having multiple datasets from multiple organizations (also called the "cross-silo" setting in federated learning) by splitting the original CIFAR-10 dataset into multiple partitions. Each partition will represent the data from a single organization. We're doing this purely for experimentation purposes, in the real world there's no need for data splitting because each organization already has their own data (so the data is naturally partitioned).

Each organization will act as a client in the federated learning system. So having ten organizations participate in a federation means having ten clients connected to the federated learning server:


In [4]:
NUM_CLIENTS = 10


Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap the resulting partitions by creating a PyTorch `DataLoader` for each of them:

In [5]:
BATCH_SIZE = 32


def load_datasets():
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets()

Files already downloaded and verified
Files already downloaded and verified


We now have a list of ten training sets and ten validation sets (`trainloaders` and `valloaders`) representing the data of ten different organizations. Each `trainloader`/`valloader` pair contains 4500 training examples and 500 validation examples. There's also a single `testloader` (we did not split the test set). Again, this is only necessary for building research or educational systems, actual federated learning systems have their data naturally distributed across multiple partitions.

Let's take a look at the first batch of images and labels in the first training set (i.e., `trainloaders[0]`) before we move on:

In [6]:
images, labels = next(iter(trainloaders[0]))

# Reshape and convert images to a NumPy array
# matplotlib requires images with the shape (height, width, 3)
images = images.permute(0, 2, 3, 1).numpy()
# Denormalize
images = images / 2 + 0.5

# Create a figure and a grid of subplots
# fig, axs = plt.subplots(4, 8, figsize=(12, 6))

# # Loop over the images and plot them
# for i, ax in enumerate(axs.flat):
#     ax.imshow(images[i])
#     ax.set_title(CLASSES[labels[i]])
#     ax.axis("off")

# # Show the plot
# fig.tight_layout()
# plt.show()

### Defining the model

We use the simple CNN described in the [PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#define-a-convolutional-neural-network):

In [7]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Let's continue with the usual training and test functions:

In [8]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Training the model

We now have all the basic building blocks we need: a dataset, a model, a training function, and a test function. Let's put them together to train the model on the dataset of one of our organizations (`trainloaders[0]`). This simulates the reality of most machine learning projects today: each organization has their own data and trains models only on this internal data: 

In [9]:
trainloader = trainloaders[0]
valloader = valloaders[0]
net = Net().to(DEVICE)

# for epoch in range(5):
#     train(net, trainloader, 1)
#     loss, accuracy = test(net, valloader)
#     print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

# loss, accuracy = test(net, testloader)
# print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

Training the simple CNN on our CIFAR-10 split for 5 epochs should result in a test set accuracy of about 41%, which is not good, but at the same time, it doesn't really matter for the purposes of this tutorial. The intent was just to show a simplistic centralized training pipeline that sets the stage for what comes next - federated learning!

### Updating model parameters

In federated learning, the server sends the global model parameters to the client, and the client updates the local model with the parameters received from the server. It then trains the model on the local data (which changes the model parameters locally) and sends the updated/changed model parameters back to the server (or, alternatively, it sends just the gradients back to the server, not the full model parameters).

We need two helper functions to update the local model with parameters received from the server and to get the updated model parameters from the local model: `set_parameters` and `get_parameters`. The following two functions do just that for the PyTorch model above.

The details of how this works are not really important here (feel free to consult the PyTorch documentation if you want to learn more). In essence, we use `state_dict` to access PyTorch model parameter tensors. The parameter tensors are then converted to/from a list of NumPy ndarray's (which Flower knows how to serialize/deserialize):

In [10]:
def get_parameters(net) -> List[np.ndarray]:
    parameters = [val.cpu().numpy() for _, val in net.state_dict().items()]
    # print(f"Parameters size: {get_params_size(parameters)} bytes")
    return parameters


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

### Implementing a Flower client

With that out of the way, let's move on to the interesting part. Federated learning systems consist of a server and multiple clients. In Flower, we create clients by implementing subclasses of `flwr.client.Client` or `flwr.client.NumPyClient`. We use `NumPyClient` in this tutorial because it is easier to implement and requires us to write less boilerplate.

To implement the Flower client, we create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`:

* `get_parameters`: Return the current local model parameters
* `fit`: Receive model parameters from the server, train the model parameters on the local data, and return the (updated) model parameters to the server
* `evaluate`: Receive model parameters from the server, evaluate the model parameters on the local data, and return the evaluation result to the server

We mentioned that our clients will use the previously defined PyTorch components for model training and evaluation. Let's see a simple Flower client implementation that brings everything together:

In [11]:
def get_params_size(parameters):
    total_size = 0
    for p in parameters:
        total_size += sys.getsizeof(p)
        if isinstance(p, np.ndarray):
            total_size += p.nbytes
    return total_size


In [12]:
import time
# import logging
# import multiprocessing

class FlowerClient(fl.client.NumPyClient):

      def __init__(self, cid, net, trainloader, valloader):
          self.cid = cid
          self.net = net
          self.trainloader = trainloader
          self.valloader = valloader
          # self.logger = logging.getLogger(f"Client {cid}")
          # self.logger.setLevel(logging.INFO)
          # self.file_handler = logging.FileHandler(f"client_{cid}_log.txt")
          # self.logger.addHandler(self.file_handler)

      def get_parameters(self, config):
          return get_parameters(self.net)

      def fit(self, parameters, config):
        with open(f"client_{self.cid}_log.txt", "a") as f:
          set_parameters(self.net, parameters)
          start_time = time.time()
          train(self.net, self.trainloader, epochs=1)
          end_time = time.time()
          total_time = end_time - start_time
          adj_total_time = ("%.3f" %(total_time))
          parameters = get_parameters(self.net)
          print(f"Client ID: {self.cid}, Parameters size: {get_params_size(parameters)} bytes, Total Time: {adj_total_time}")
          f.write(f"Client ID: {self.cid}, Parameters size: {get_params_size(parameters)} bytes, Total Time: {adj_total_time}\n")
          f.write("\n")
          return parameters, len(self.trainloader), {}

      def evaluate(self, parameters, config):
        # with open(f"client_{self.cid}_evaluate.txt", "w") as d:
        with open(f"client_{self.cid}_evaluate.txt", "a") as f:
          set_parameters(self.net, parameters)
          loss, accuracy = test(self.net, self.valloader)
          print(f"Client ID: {self.cid}, Loss: {loss}, Accuracy: {accuracy}")
          f.write(f"Client ID: {self.cid}, Loss: {loss}, Accuracy: {accuracy}")
          f.write("\n")
          return float(loss), len(self.valloader), {"accuracy": float(accuracy)}



Our class `FlowerClient` defines how local training/evaluation will be performed and allows Flower to call the local training/evaluation through `fit` and `evaluate`. Each instance of `FlowerClient` represents a *single client* in our federated learning system. Federated learning systems have multiple clients (otherwise, there's not much to federate), so each client will be represented by its own instance of `FlowerClient`. If we have, for example, three clients in our workload, then we'd have three instances of `FlowerClient`. Flower calls `FlowerClient.fit` on the respective instance when the server selects a particular client for training (and `FlowerClient.evaluate` for evaluation).

### Using the Virtual Client Engine

In this notebook, we want to simulate a federated learning system with 10 clients on a single machine. This means that the server and all 10 clients will live on a single machine and share resources such as CPU, GPU, and memory. Having 10 clients would mean having 10 instances of `FlowerClient` in memory. Doing this on a single machine can quickly exhaust the available memory resources, even if only a subset of these clients participates in a single round of federated learning.

In addition to the regular capabilities where server and clients run on multiple machines, Flower, therefore, provides special simulation capabilities that create `FlowerClient` instances only when they are actually necessary for training or evaluation. To enable the Flower framework to create clients when necessary, we need to implement a function called `client_fn` that creates a `FlowerClient` instance on demand. Flower calls `client_fn` whenever it needs an instance of one particular client to call `fit` or `evaluate` (those instances are usually discarded after use, so they should not keep any local state). Clients are identified by a client ID, or short `cid`. The `cid` can be used, for example, to load different local data partitions for different clients, as can be seen below:

In [13]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

### Starting the training

We now have the class `FlowerClient` which defines client-side training/evaluation and `client_fn` which allows Flower to create `FlowerClient` instances whenever it needs to call `fit` or `evaluate` on one particular client. The last step is to start the actual simulation using `flwr.simulation.start_simulation`. 

The function `start_simulation` accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate (`num_clients`), the number of federated learning rounds (`num_rounds`), and the strategy. The strategy encapsulates the federated learning approach/algorithm, for example, *Federated Averaging* (FedAvg).

Flower has a number of built-in strategies, but we can also use our own strategy implementations to customize nearly all aspects of the federated learning approach. For this example, we use the built-in `FedAvg` implementation and customize it using a few basic parameters. The last step is the actual call to `start_simulation` which - you guessed it - starts the simulation:

In [14]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [15]:
#function to remove old files (possibly for colab only*)

import os
import glob

file_key = "client*.txt"
for del_file in glob.glob(file_key):
  os.remove(del_file)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

The only thing left to do is to tell the strategy to call this function whenever it receives evaluation metric dictionaries from the clients:

In [16]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10,
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources,
)



INFO flwr 2023-04-27 20:10:13,750 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-27 20:10:17,182	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-04-27 20:10:18,536 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3861033369.0, 'node:172.28.0.12': 1.0, 'memory': 7722066740.0, 'GPU': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3861033369.0, 'node:172.28.0.12': 1.0, 'memory': 7722066740.0, 'GPU': 1.0, 'CPU': 2.0}
INFO flwr 2023-04-27 20:10:18,544 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-27 20:10:18,550 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
INFO flwr 2023-04-27 20:10:31,609 | se

(launch_and_fit pid=58330) Client ID: 3, Parameters size: 249288 bytes, Total Time: 2.110
(autoscaler +39s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +39s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(launch_and_fit pid=58409) Client ID: 9, Parameters size: 249288 bytes, Total Time: 1.888
(launch_and_fit pid=58483) Client ID: 0, Parameters size: 249288 bytes, Total Time: 1.893
(launch_and_fit pid=58571) Client ID: 7, Parameters size: 249288 bytes, Total Time: 2.607
(launch_and_fit pid=58649) Client ID: 8, Parameters size: 249288 bytes, Total Time: 1.884
(launch_and_fit pid=58729) Client ID: 6, Parameters size: 249288 bytes, Total Time: 3.342
(launch_and_fit pid=58814) Client ID: 5, Parameters size: 249288 bytes, Tot

DEBUG flwr 2023-04-27 20:12:11,899 | server.py:232 | fit_round 1 received 10 results and 0 failures
DEBUG:flwr:fit_round 1 received 10 results and 0 failures
WARNING flwr 2023-04-27 20:12:11,927 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-04-27 20:12:11,930 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 5 clients (out of 10)


(launch_and_fit pid=59057) Client ID: 4, Parameters size: 249288 bytes, Total Time: 2.781
(autoscaler +2m10s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(launch_and_evaluate pid=59163) Client ID: 8, Loss: 0.06437034440040589, Accuracy: 0.29
(launch_and_evaluate pid=59227) Client ID: 1, Loss: 0.06371496081352233, Accuracy: 0.28
(launch_and_evaluate pid=59301) Client ID: 5, Loss: 0.06228993773460388, Accuracy: 0.29
(launch_and_evaluate pid=59366) Client ID: 3, Loss: 0.06419980239868164, Accuracy: 0.262


DEBUG flwr 2023-04-27 20:12:51,670 | server.py:182 | evaluate_round 1 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2023-04-27 20:12:51,675 | server.py:218 | fit_round 2: strategy sampled 10 clients (out of 10)
DEBUG:flwr:fit_round 2: strategy sampled 10 clients (out of 10)


...

DEBUG flwr 2023-04-27 20:21:35,087 | server.py:232 | fit_round 5 received 10 results and 0 failures
DEBUG:flwr:fit_round 5 received 10 results and 0 failures
DEBUG flwr 2023-04-27 20:21:35,122 | server.py:168 | evaluate_round 5: strategy sampled 5 clients (out of 10)
DEBUG:flwr:evaluate_round 5: strategy sampled 5 clients (out of 10)


(launch_and_fit pid=63900) Client ID: 9, Parameters size: 249288 bytes, Total Time: 2.736
(launch_and_evaluate pid=64008) Client ID: 3, Loss: 0.049455974340438844, Accuracy: 0.432
(autoscaler +11m41s) Warning: The following resource request cannot be scheduled right now: {'GPU': 1.0, 'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(launch_and_evaluate pid=64076) Client ID: 2, Loss: 0.0505025806427002, Accuracy: 0.414
(launch_and_evaluate pid=64148) Client ID: 6, Loss: 0.04890633201599121, Accuracy: 0.436
(launch_and_evaluate pid=64213) Client ID: 7, Loss: 0.04780472278594971, Accuracy: 0.462


DEBUG flwr 2023-04-27 20:22:16,437 | server.py:182 | evaluate_round 5 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 5 received 5 results and 0 failures
INFO flwr 2023-04-27 20:22:16,444 | server.py:147 | FL finished in 704.8255090269995
INFO:flwr:FL finished in 704.8255090269995
INFO flwr 2023-04-27 20:22:16,447 | app.py:218 | app_fit: losses_distributed [(1, 0.06353171119689942), (2, 0.05665622801780701), (3, 0.0522362476825714), (4, 0.05039014592170715), (5, 0.04931559190750122)]
INFO:flwr:app_fit: losses_distributed [(1, 0.06353171119689942), (2, 0.05665622801780701), (3, 0.0522362476825714), (4, 0.05039014592170715), (5, 0.04931559190750122)]
INFO flwr 2023-04-27 20:22:16,449 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-04-27 20:22:16,451 | app.py:220 | app_fit: metrics_distributed {'accuracy': [(1, 0.28080000000000005), (2, 0.34680000000000005), (3, 0.40199999999999997), (4, 0.42400000000000004), (5, 

History (loss, distributed):
	round 1: 0.06353171119689942
	round 2: 0.05665622801780701
	round 3: 0.0522362476825714
	round 4: 0.05039014592170715
	round 5: 0.04931559190750122
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.28080000000000005), (2, 0.34680000000000005), (3, 0.40199999999999997), (4, 0.42400000000000004), (5, 0.42919999999999997)]}

(launch_and_evaluate pid=64291) Client ID: 0, Loss: 0.04990834975242615, Accuracy: 0.402


In [17]:

file_key = "client*.txt"
file_name = input("Enter file name with no .txt extension: ")
input_file_name = file_name + '.txt'
with open(input_file_name, "w") as outfile:
  for file in glob.glob(file_key):
    with open(file, 'r') as infile:
      outfile.write(infile.read())
      outfile.write("\n")


Enter file name with no .txt extension: output
